# Hubert Stefanski - Data Mining II - Main Assignment
## 20081102@mail.wit.ie

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder

from IPython.display import display, Markdown
plt.style.use("seaborn-darkgrid")
pd.set_option('display.max_columns', None)  

## Helper functions
don't mind this, I'm just lazy

In [2]:

def name_func(df):
    col_names = []
    for i in range(df.shape[1]):
        col_names.append("Col_" + str(i))
    return col_names

## Load Datasets

In [3]:
# Load and name columns, change last column name to "Target"
df_labeled = pd.read_csv("data/labeled.csv.gz",names = name_func(pd.read_csv("data/labeled.csv.gz")))
df_labeled.columns = [*df_labeled.columns[:-1],"Target"]
print("Labeled", df_labeled.shape)
# df_labeled.head()

# Load and name columns
df_scoring = pd.read_csv("data/scoring.csv.gz",names = name_func(pd.read_csv("data/scoring.csv.gz")))
print("Scoring", df_scoring.shape)
# df_scoring.head()
                         

Labeled (16000, 177)
Scoring (9000, 176)


## Cleanup

In [4]:
# Change categorical data to binary flag
for col in df_labeled.select_dtypes(include=['object']).copy():
    df_labeled[col] = df_labeled[col].str.lower()
    encoder = OrdinalEncoder(categories=[['no','yes']])
    df_labeled[col] = encoder.fit_transform(df_labeled.loc[:,[col]])

# Check whether NaNs exist
print("Nan exists?: " + str(df_labeled.isnull().values.any()))

# Check total number of NaNs
print("NaN count: " + str(df_labeled.isnull().sum().sum()))

# Cleanup NaNs
for col in df_labeled.columns:
    if df_labeled[col].isna().values.any():
        # Replace all NaNs with means of the column
        df_labeled[col] = df_labeled[col].replace(np.nan, df_labeled[col].mean())

# Check whether NaNs exist after nan Cleanup      
print("Nan exists?: " + str(df_labeled.isnull().values.any()))


Nan exists?: True
NaN count: 3365
Nan exists?: False


## Output the cleaned up DataFrame to a fresh csv
This allows some insight through dTale

In [5]:
df_labeled.to_csv("output/clean_df_labeled.csv", index=False)